In [1]:
from ToxicMl.trainer import GenericRegressionTrainer
from ToxicMl.metrics import MAE, MSE, MaxError

import torch
from ToxicMl.dataset import LipoDataset

import torch
from pathlib import Path

root = Path("dataset/ogbg_mollipo_custom")
EPOCHS = 8

In [2]:
dataset = LipoDataset(root)
split_idx = dataset.get_idx_split() 
train = dataset[split_idx["train"]]
validation = dataset[split_idx["valid"]]
test = dataset[split_idx["test"]]

train_metrics = [MAE(), MSE(), MaxError()]
validation_metrics = [MAE(), MSE(), MaxError()]
EPOCHS = 10

In [8]:
from ToxicMl.MLmodels.gcn import ChemConvBlock
import torch
import torch.nn.functional as F

from torch.nn import Linear, ReLU
from torch_geometric.data import Data
from torch_geometric.nn.pool import global_max_pool

from torch.nn import BatchNorm1d

class ChemConvDescriptorsReg(torch.nn.Module):
    def __init__(self, embedder_depth, in_channels, hidden_channels, out_channels, in_descriptors):
        super().__init__()
        self.in_descriptors = in_descriptors
        self.encoder = torch.nn.ModuleList([
            ChemConvBlock(in_channels=in_channels, out_channels=hidden_channels)
            if i == 0
            else ChemConvBlock(in_channels=hidden_channels, out_channels=hidden_channels)
            for i in range(embedder_depth)])
        self.norm1 = BatchNorm1d(hidden_channels+in_descriptors)
        self.linear1 = Linear(hidden_channels+in_descriptors, 64)
        self.relu1 = ReLU(inplace=True)
        self.linear2 = Linear(64, 32)
        self.relu2 = ReLU(inplace=True)
        self.linear3 = Linear(32, out_channels)
        self.relu3 = ReLU(inplace=True)
        

    def forward(self, data: Data):
        for module in self.encoder:
            data = module(data)
        x = global_max_pool(data.x, data.batch)


        d = data.descriptors.view(-1, self.in_descriptors)
        x = torch.cat([x,d], dim=1)
        x = self.norm1(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        return x
    

In [10]:
from ToxicMl.MLmodels.gcn import GCNReg, ChemGCNReg
model = ChemConvDescriptorsReg(3, 133, 16, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 3-16, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:00<00:00, 157.59it/s]
val epoch:   0%|          | 0/14.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 14/14.0

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▂▃▂▁▂▂▁▁▂
train/loss,█▁▁▂▂▂▂▂▃▃
validation/MAE,▄▄▃▅█▄▁▅▂▂
validation/MSE,▁█▇██▇▆█▆▇
validation/Max Error,▁█▆▇▆▇▆▇▆▇
validation/loss,▁▁▁▁▁▁█▁▁▁
train/MAE,0.97885
train/MSE,1.48639
train/Max Error,4.72509


In [11]:
model = ChemConvDescriptorsReg(3, 133, 32, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 3-32, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:00<00:00, 134.65it/s]
val epoch:   0%|          | 0/14.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 14/14.0

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▁▁▂▁▁▁▁▁▂
train/loss,█▂▂▄▂▃▁▁▁▃
validation/MAE,▁▇▅▆▆▇▇▇█▅
validation/MSE,▁██▇▇███▇▇
validation/Max Error,▁██▇▇███▇█
validation/loss,▂▂▃█▂▄▁▄▂▁
train/MAE,0.99053
train/MSE,1.52832
train/Max Error,4.16707


In [12]:
model = ChemConvDescriptorsReg(3, 133, 64, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 3-64, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:00<00:00, 109.18it/s]
val epoch:   0%|          | 0/14.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 14/14.0

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▃▂▂▂▃▁▂▂▁
train/loss,█▂▁▂▂▃▁▂▁▂
validation/MAE,▇▄▄▆▄▆▄▁█▄
validation/MSE,▁▇███▆▆▄█▇
validation/Max Error,▁█▇▇▇▅▅▄▄▇
validation/loss,█▃▃▁▄▅▂▃▄█
train/MAE,0.99044
train/MSE,1.50873
train/Max Error,3.89094


In [13]:
model = ChemConvDescriptorsReg(5, 133, 16, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 5-16, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:00<00:00, 112.73it/s]
val epoch:   0%|          | 0/14.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 14/14.0

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▁▁▁▁▂▂▂▂▁
train/loss,█▁▄▃▂▄▃▄▃▂
validation/MAE,▆█▁▂▇▅▅▆▁▆
validation/MSE,▂█▃▁▅▄▃▄▃▅
validation/Max Error,▂█▄▁▃▄▂▄▄▅
validation/loss,▁▂▃▁█▅▂▃▄▂
train/MAE,0.98207
train/MSE,1.49246
train/Max Error,4.52619


In [14]:
model = ChemConvDescriptorsReg(5, 133, 32, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 5-32, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:00<00:00, 105.83it/s]
val epoch:   0%|          | 0/14.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 14/14.0

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▂▂▂▂▂▁▂▃▂
train/loss,█▁▂▄▂▂▂▃▂▁
validation/MAE,▁▅▅▆▅▆█▅▆▃
validation/MSE,▁▅▅▅▄▅█▄▅▄
validation/Max Error,▁██▇█▇▇█▇▇
validation/loss,▁▁▁▁█▁▁▁▁▁
train/MAE,0.98279
train/MSE,1.49421
train/Max Error,4.60001


In [15]:
model = ChemConvDescriptorsReg(5, 133, 64, 1, 200)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericRegressionTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    None,
    EPOCHS,
    None,
    32
)

trainer.train("LIPO GCN 5-64, custom dataset, descriptor")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/105.0 [00:00<?, ?it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train epoch: 100%|██████████| 105/105.0 [00:01<00:00, 92.42it/s]
val epoch:  57%|█████▋    | 8/14.0 [00:00<00:00, 78.96it/s]/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
val epoch: 100%|██████████| 

train/MAE,█▁▁▁▁▁▁▁▁▁
train/MSE,█▁▁▁▁▁▁▁▁▁
train/Max Error,█▃▂▁▂▂▃▃▂▃
train/loss,█▁▃▇▄▂▅▁▄▂
validation/MAE,▂▂▂▁▅█▃▃▅▇
validation/MSE,▁█▆▆▆█▇▇▇▆
validation/Max Error,▁█▇▇▇▇▇▇▇▇
validation/loss,▄▃▆█▅▂▁▄▇▇
train/MAE,0.9834
train/MSE,1.50239
train/Max Error,4.85438
